# Multi-agent supervisor

1. Research와 math agents를 빌드
2. prebuilt "langgraph-supervisor"를 이용해서 supervisor를 빌드

### 1.Worker Agent 만들기

- Research agent : Tavily API를 이용해서 웹서치를 수행
- Math agent : simple math tools를 이용

In [1]:
from dotenv import load_dotenv

load_dotenv()


True

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search = TavilySearchResults(max_results=4)
web_search_result = web_search.invoke("What is the capital of France?")

print(web_search_result)


[{'title': 'List of capitals of France - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/List_of_capitals_of_France', 'content': 'Find sources:\xa0"List of capitals of France"\xa0–\xa0news\xa0· newspapers\xa0· books\xa0· scholar\xa0· JSTOR (July 2012) (Learn how and when to remove this message)\nThis is a chronological list of capitals of France. The capital of France has been Paris since its liberation in 1944.[1]\nChronology[edit]\nTournai (before 486), current-day Belgium\nSoissons (486–936)\nLaon (936–987)\nParis (987–1419), the residence of the Kings of France, although they were consecrated at Reims. [...] Bordeaux (September 1914), the government was relocated from Paris to Bordeaux very briefly during World War I, when it was feared that Paris would soon fall into German hands. These fears were alleviated after the German Army was pushed back at the First Battle of the Marne.\nTours (10–13 June 1940), the city served as the temporary capital of France during World War II afte

In [3]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
prompt_research = """
        "You are a research agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with research-related tasks, DO NOT do any math\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
"""

research_agent = create_react_agent(
    model=llm,
    tools=[web_search],
    prompt=prompt_research,
    name = "research_agent",
)

In [4]:

list_chunk = list()


for chunk in research_agent.stream(
    {"messages": [{"role": "user", "content": "who is the mayor of NYC"}]}
):
    list_chunk.append(chunk)
    print(chunk)


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ItMnzRXSr8ZVZORUiZCPLa5g', 'function': {'arguments': '{"query":"current mayor of New York City 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 151, 'total_tokens': 178, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'id': 'chatcmpl-BT6f4i8FU1YFxHE0bWQfKte5VFVNm', 'finish_reason': 'tool_calls', 'logprobs': None}, name='research_agent', id='run-9bdb7d61-c671-43e9-89c5-ce81df191bf5-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current mayor of New York City 2023'}, 'id': 'call_ItMnzRXSr8ZVZORUiZCPLa5g', 'type': 'tool_call

In [5]:
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

In [6]:
for chunk in list_chunk:
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: research_agent
Tool Calls:
  tavily_search_results_json (call_ItMnzRXSr8ZVZORUiZCPLa5g)
 Call ID: call_ItMnzRXSr8ZVZORUiZCPLa5g
  Args:
    query: current mayor of New York City 2023


Update from node tools:


================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Eric Adams - Wikipedia", "url": "https://en.wikipedia.org/wiki/Eric_Adams", "content": "110thMayor of New York City\nIncumbent\nAssumed officeJanuary 1, 2022\nDeputy | Lorraine Grillo(2022)Sheena Wright(2023–2024)Maria Torres-Springer(2024–2025)Camille Joseph Varlack(2025; interim)[1][2]Randy Mastro(2025–present)\nPreceded by | Bill de Blasio\n18thBorough PresidentofBrooklyn\nIn officeJanuary 1, 2014 – December 31, 2021\nDeputy | Diana ReynaIngrid Lewis-Martin\nPreceded by | Marty Markowitz\nSucceeded by | Antonio Reynoso\nMember of theNew Yor

In [7]:
def add(a: float, b: float):
    """Add two numbers."""
    return a + b


def multiply(a: float, b: float):
    """Multiply two numbers."""
    return a * b


def divide(a: float, b: float):
    """Divide two numbers."""
    return a / b

llm_math = ChatOpenAI(model="gpt-4o-mini")
prompt_math = """
        "You are a math agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with math-related tasks\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
"""


math_agent = create_react_agent(
    model=llm_math,
    tools=[add, multiply, divide],
    prompt=prompt_math,
    name="math_agent"
)


In [8]:
for chunk in math_agent.stream(
    {"messages": [{"role": "user", "content": "what's (3 + 5) x 7"}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: math_agent
Tool Calls:
  add (call_FFoHSOzyu0TcXxLYdOeMS4Wg)
 Call ID: call_FFoHSOzyu0TcXxLYdOeMS4Wg
  Args:
    a: 3
    b: 5


Update from node tools:


================================= Tool Message =================================
Name: add

8.0


Update from node agent:


================================== Ai Message ==================================
Name: math_agent
Tool Calls:
  multiply (call_CWTpRiNeRyn23GjPvAal6Vlj)
 Call ID: call_CWTpRiNeRyn23GjPvAal6Vlj
  Args:
    a: 8
    b: 7


Update from node tools:


================================= Tool Message =================================
Name: multiply

56.0


Update from node agent:


================================== Ai Message ==================================
Name: math_agent

56.0




## 2. langgraph-supervisor를 이용해서 supervisor 만들기

In [10]:
from langgraph_supervisor import create_supervisor

prompt_supervisor = """
        "You are a supervisor managing two agents:\n"
        "- a research agent. Assign research-related tasks to this agent\n"
        "- a math agent. Assign math-related tasks to this agent\n"
        "Assign work to one agent at a time, do not call agents in parallel.\n"
        "Do not do any work yourself."
"""

llm_supervisor = ChatOpenAI(model="gpt-4.1")

supervisor = create_supervisor(
    model=llm_supervisor,
    agents=[research_agent, math_agent],
    prompt=prompt_supervisor,
    add_handoff_back_messages=True,
    output_mode="full_history"
)

"""
supervisor로 제어가 다시 돌아올 때(즉, handoff가 발생했음을 나타내기 위해),
메시지 히스토리에 (AIMessage, ToolMessage) 쌍을 추가할지 여부를 결정하는 옵션
"""

'\nsupervisor로 제어가 다시 돌아올 때(즉, handoff가 발생했음을 나타내기 위해),\n메시지 히스토리에 (AIMessage, ToolMessage) 쌍을 추가할지 여부를 결정하는 옵션\n'

## 3. 직접 Supervisor 만들기

Set up agent communication

In [ ]:
from typing import Annotated
from langchain_core.tools import tool, InjectedToolCallId